In [20]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import warnings
import json

warnings.filterwarnings('ignore')


nf_data_path = r'../../../Data/noFinacialFeatures.csv'
f_data_path = r'../../../Data/FinacialFeatures.csv'

In [10]:
nf_df = pd.read_csv(nf_data_path)
f_df = pd.read_csv(f_data_path)

In [11]:
df = pd.DataFrame()
df = nf_df.copy()
f_df.drop(['企业总评分'], axis=1, inplace=True)
df = df.merge(f_df, how='left', on='企业编号')

In [12]:
y = df[['企业编号', '企业总评分']]
x = df.drop(['企业总评分'], axis=1)

xtrain, xtest, ytrain, ytest = train_test_split(
    x, y, test_size=0.2, random_state=0)
ytrain_id = ytrain['企业编号']
ytrain = ytrain['企业总评分']
ytest_id = ytest['企业编号']
ytest = ytest['企业总评分']
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

id_train = xtrain['企业编号']
id_test = xtest['企业编号']
xtrain.drop(['企业编号'], axis=1, inplace=True)
xtest.drop(['企业编号'], axis=1, inplace=True)

# Normal use

In [21]:
xgb = XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

In [22]:
traindf, testdf = train_test_split(xtrain, test_size = 0.3)
xgb.fit(xtrain,ytrain)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.08, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.75)

In [25]:
pred = xgb.predict(xtest)

In [27]:
np.sqrt(mse(pred.astype(int), ytest.astype(int)))

3.239849012164243

# Cross Validation

In [15]:
K = 10
kf = KFold(n_splits=K, random_state=3228, shuffle=True)

In [ ]:
xgb_params = {'eta': 0.02, 
              'max_depth': 4, 
              'subsample': 0.9, 
              'colsample_bytree': 0.9,
              'objective': 'binary:logistic',
              'eval_metric': 'rmse', 
              'seed': 99, 
              'silent': True}

In [18]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(xtrain.values, ytrain.values)):
    print('----')
    print("fold n°{}".format(fold_))
    
    train_X, valid_X = xtrain.iloc[trn_idx], ytrain.iloc[trn_idx]
    train_y, valid_y = xtrain.iloc[val_idx], ytrain.iloc[val_idx]

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(xtest)

    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))